### Importing Libraries 

In [0]:
from nltk.tokenize import word_tokenize
import nltk
import re
import gensim
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import keras.backend as K
from itertools import chain
from keras.utils import to_categorical
from keras.callbacks import *
from keras.optimizers import *
import matplotlib.pyplot as plt
from keras.initializers import *
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from keras.engine.topology import Layer
from keras.models import Sequential,Model
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
from keras.preprocessing.sequence import pad_sequences
from keras import initializers, regularizers, constraints
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Dropout, AveragePooling1D, LSTM, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Flatten, GRU, Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
#from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score
import matplotlib.pyplot as plt
from keras.layers import concatenate,dot
from keras.preprocessing import sequence
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
#from skmultilearn.problem_transform import LabelPowerset

### Monting Google Drive **folders**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Files **Reading**

In [0]:
train=pd.read_csv("/content/drive/My Drive/Fourth Semester/Ai/AI Project/train all features /trainAllfeatures.csv")
test = pd.read_csv("/content/drive/My Drive/Fourth Semester/Ai/AI Project/test all features/testAllfeatures.csv")
trainsample= train.drop(['PatientID'],axis=1)
testsample= test.drop(['PatientID'],axis=1)

### Showing the file content (After dropping the ID coulmn)


In [5]:
trainsample

,age,target,Event,Mstage,Nstage,Tstage,original_shape_Compactness1,original_shape_Compactness2,original_shape_Maximum3DDiameter,original_shape_SphericalDisproportion,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_Kurtosis,original_firstorder_Maximum,original_firstorder_Mean,original_firstorder_MeanAbsoluteDeviation,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_StandardDeviation,original_firstorder_Uniformity,original_firstorder_Variance,original_glcm_Autocorrelation,original_glcm_ClusterProminence,original_glcm_ClusterShade,original_glcm_ClusterTendency,original_glcm_Contrast,original_glcm_Correlation,original_glcm_DifferenceEntropy,original_glcm_DifferenceAverage,original_glcm_JointEnergy,original_glcm_JointEntropy,original_glcm_Id,original_glcm_Idm,original_glcm_Imc1,original_glcm_Imc2,original_glcm_Idmn,original_glcm_Idn,original_glcm_InverseVariance,original_glcm_MaximumProbability,original_glcm_SumAverage,original_glcm_SumEntropy,original_glrlm_ShortRunEmphasis,original_glrlm_LongRunEmphasis,original_glrlm_GrayLevelNonUniformity,original_glrlm_RunLengthNonUniformity,original_glrlm_RunPercentage,original_glrlm_LowGrayLevelRunEmphasis,original_glrlm_HighGrayLevelRunEmphasis,original_glrlm_ShortRunLowGrayLevelEmphasis,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glrlm_LongRunHighGrayLevelEmphasis
0,66.0000,1378,0,0,0,2,0.027815,0.274892,48.559242,1.537964,0.650210,5431.333210,0.275228,19786,3942944085,5.138062,1.592466,241,-310.858031,288.311105,-266.0,-1006,1247,446.407319,-0.281714,320.385361,0.037008,102646.779500,1056.843506,546253.499800,-5334.563631,555.818455,54.336816,0.821101,3.845383,5.036775,0.005920,9.270194,0.338672,0.262780,-0.175185,0.901525,0.981053,0.917509,0.249416,0.037768,61.035662,6.007130,0.926391,1.457980,555.378594,14592.303730,0.891330,0.003562,954.749136,0.003384,851.987373,0.004444,1651.710761
1,64.5722,379,1,0,2,4,0.023015,0.188210,75.703368,1.744961,0.573079,10369.568730,0.240727,43168,3122890068,4.461054,3.371013,262,-134.429809,191.395154,-15.0,-945,1207,268.966000,-1.216062,232.962090,0.080061,54271.335560,1219.689108,331155.230400,-6857.579704,295.032309,16.868781,0.891184,2.984034,2.570017,0.025532,7.551168,0.495719,0.439879,-0.250370,0.932847,0.993282,0.953464,0.340024,0.110575,67.826804,5.255594,0.855154,2.512125,1766.619373,22881.932690,0.766032,0.002246,1092.823724,0.002069,884.838627,0.003633,3386.995589
2,66.0452,573,1,0,3,2,0.027348,0.265740,70.434367,1.555420,0.642913,10558.818690,0.200766,52655,1059535256,3.437111,7.585494,115,-54.136093,94.132136,2.0,-785,900,141.852795,-2.153344,131.116356,0.160905,17191.498870,972.465798,54523.086870,-1618.363118,78.183970,7.040623,0.831081,2.333939,1.496020,0.066569,5.686554,0.616607,0.581458,-0.249969,0.884911,0.995095,0.963887,0.379957,0.189723,61.795144,4.159019,0.769265,4.096838,3462.816344,18773.106000,0.644554,0.001880,878.989601,0.001635,634.910036,0.004869,4232.418080
3,59.3566,959,0,0,2,3,0.026811,0.255406,46.818800,1.576120,0.634469,4221.412123,0.323878,13074,658332446,4.352564,4.281865,274,-118.705522,149.072998,-37.0,-850,1124,224.397694,-1.437836,190.429315,0.074476,36263.324020,968.117567,166735.780000,-3968.921514,184.299591,13.656782,0.860096,2.926791,2.492079,0.015048,7.536161,0.457545,0.391681,-0.214233,0.902811,0.993509,0.950374,0.357236,0.050439,60.841486,5.122729,0.880123,1.824328,683.869792,7978.285176,0.828138,0.002855,907.045929,0.002680,774.885387,0.003901,1844.746046
4,71.0554,2119,0,0,3,4,0.023691,0.199424,53.795911,1.711620,0.584242,5295.900331,0.327241,16237,807253170,4.055700,8.999616,203,-67.201946,143.210231,14.0,-1023,1226,222.972850,-2.438490,212.604774,0.104419,45200.790100,1594.558416,658805.554000,-10

In [6]:
testsample

,age,Mstage,Nstage,Tstage,original_shape_Compactness2,original_shape_Maximum3DDiameter,original_shape_SphericalDisproportion,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_Kurtosis,original_firstorder_Maximum,original_firstorder_Mean,original_firstorder_MeanAbsoluteDeviation,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_StandardDeviation,original_firstorder_Uniformity,original_firstorder_Variance,original_glcm_Autocorrelation,original_glcm_ClusterProminence,original_glcm_ClusterShade,original_glcm_ClusterTendency,original_glcm_Contrast,original_glcm_Correlation,original_glcm_DifferenceEntropy,original_glcm_DifferenceAverage,original_glcm_JointEnergy,original_glcm_JointEntropy,original_glcm_Id,original_glcm_Idm,original_glcm_Imc1,original_glcm_Imc2,original_glcm_Idmn,original_glcm_Idn,original_glcm_InverseVariance,original_glcm_MaximumProbability,original_glcm_SumAverage,original_glcm_SumEntropy,original_glrlm_ShortRunEmphasis,original_glrlm_LongRunEmphasis,original_glrlm_GrayLevelNonUniformity,original_glrlm_RunLengthNonUniformity,original_glrlm_RunPercentage,original_glrlm_LowGrayLevelRunEmphasis,original_glrlm_HighGrayLevelRunEmphasis,original_glrlm_ShortRunLowGrayLevelEmphasis,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glrlm_LongRunHighGrayLevelEmphasis
0,44.3970,0,0,4,0.296451,106.901824,1.499738,0.666783,29085.541430,0.114528,254069,2006942430,3.073901,14.142104,765,16.933707,44.923384,19.0,-654,1419,88.877457,-0.701919,87.249366,0.201986,7612.451871,810.133656,21777.73407,-49.637453,38.349991,2.733885,0.865405,1.799077,0.914393,0.082816,5.048950,0.687668,0.666545,-0.306548,0.906520,0.999197,0.984980,0.417331,0.193666,56.611982,3.872300,0.686184,5.786839,21219.447220,64776.697580,0.567095,0.001493,808.353957,0.001094,554.111998,0.007535,4654.031976
1,63.3183,0,3,1,0.362660,18.814888,1.402276,0.713126,629.443640,0.703879,911,262393816,5.206990,2.057162,60,-479.029638,204.396300,-479.0,-992,1052,536.682716,0.041489,241.989554,0.028924,58558.944240,579.895336,229171.44570,-765.995686,325.752485,49.740401,0.734499,3.803408,5.612222,0.002095,9.139372,0.257633,0.165967,-0.244386,0.957683,0.974940,0.891016,0.173590,0.006783,45.199122,6.059859,0.966275,1.160397,25.291608,795.675280,0.953475,0.011646,548.964573,0.010836,529.644029,0.015633,639.621963
2,64.7255,0,2,2,0.090945,105.080921,2.223687,0.449704,12509.265440,0.315298,39809,665032231,3.262327,17.726780,169,-23.682308,76.920794,19.0,-1019,1188,129.250048,-3.433133,127.061887,0.177325,16144.723180,1685.733219,145633.06590,-2863.528576,84.995985,7.890843,0.829166,2.250888,1.428105,0.070042,5.533082,0.633986,0.603222,-0.241592,0.873649,0.996769,0.973099,0.392381,0.192881,81.644318,4.055586,0.763929,4.064385,3225.808332,14007.509280,0.644247,0.001171,1600.952265,0.001003,1181.377774,0.003027,7156.937667
3,65.3635,0,0,2,0.349373,46.968074,1.419832,0.704309,4067.657372,0.282104,14458,375428731,3.589861,14.145887,422,-31.550768,91.616295,15.0,-990,1412,161.142331,-3.133562,158.023416,0.138235,24971.400000,1582.584241,275939.43230,-5124.093237,139.276676,12.627536,0.832694,2.489973,1.859968,0.039920,6.253236,0.575949,0.538017,-0.239166,0.890556,0.996313,0.970611,0.414928,0.118059,78.763414,4.470239,0.791925,2.902586,1147.928170,6094.805584,0.711467,0.002104,1533.217788,0.001900,1182.876363,0.003685,4776.549408
4,50.0000,0,0,2,0.112759,56.542020,2.069901,0.483115,7093.365690,0.376032,18942,9108915274,5.067418,3.645805,336,-648.072748,194.227188,-720.0,-1024,1360,693.458403,1.108723,246.751433,0.037058,60886.269550,327.830300,414004.33490,6575.989794,338.199701,72.888824,0.646720,4.129335,6.290405,0.001831,9.813896,0.262449,0.175369,-0.087334,0.750815,0.978024,0.904510,0.180

### Display new class counts

In [0]:
trainsample.target.value_counts().sort_index()
#x=sample.loc[:, sample.columns != 'target']
train, test = train_test_split(trainsample, test_size=.33, random_state=7,shuffle=True)
#y_train = to_categorical(train.target,num_classes=3)
#y_test = to_categorical(test.target,num_classes=3)
y_train =train.target
y_test =test.target

### Showing the rows and columns shape

In [9]:
train.shape, test.shape, y_train.shape, y_test.shape

((201, 59), (99, 59), (201,), (99,))

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict

In [11]:
train_attributes=train.columns.values[2:202]
train_correlations=train[train_attributes].corr().abs().unstack().sort_values(kind='quicksort').reset_index()
train_correlations=train_correlations[train_correlations['level_0']!=train_correlations['level_1']]
train_correlations.head(10)

,level_0,level_1,0
0,Mstage,original_glcm_Imc1,0.000429
1,original_glcm_Imc1,Mstage,0.000429
2,original_shape_Sphericity,original_glrlm_ShortRunLowGrayLevelEmphasis,0.000839
3,original_glrlm_ShortRunLowGrayLevelEmphasis,original_shape_Sphericity,0.000839
4,original_shape_Compactness1,original_glrlm_ShortRunLowGrayLevelEmphasis,0.001317
5,original_glrlm_ShortRunLowGrayLevelEmphasis,original_shape_Compactness1,0.001317
6,original_glrlm_GrayLevelNonUniformity,original_glcm_ClusterShade,0.002284
7,original_glcm_ClusterShade,original_glrlm_GrayLevelNonUniformity,0.002284
8,original_glrlm_RunLengthNonUniformity,Mstage,0.002513
9,Mstage,original_glrlm_RunLengthNonUniformity,0.002513


### Gradient Boosting with DTC

In [12]:
gradientBoosting = RandomForestClassifier(criterion = "gini", 
                                       max_depth= 6,
                                       n_estimators=100, 
                                       max_features='log2', 
                                       )
gradientBoosting.fit(train, y_train)
Y_pred = gradientBoosting.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00       1.0
          21       0.00      0.00      0.00       0.0
          24       0.00      0.00      0.00       1.0
          25       0.00      0.00      0.00       0.0
          33       0.00      0.00      0.00       1.0
          43       0.00      0.00      0.00       1.0
          50       0.00      0.00      0.00       0.0
          65       0.00      0.00      0.00       1.0
          66       0.00      0.00      0.00       1.0
          76       0.00      0.00      0.00       0.0
          77       0.00      0.00      0.00       1.0
          79       0.00      0.00      0.00       0.0
          85       0.00      0.00      0.00       1.0
          87       0.00      0.00      0.00       1.0
          98       0.00      0.00      0.00       1.0
         100       0.00      0.00      0.00       1.0
         120       0.00      0.00      0.00       1.0
         130       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Gradient Boosting Classifier


In [0]:
gradientBoosting = GradientBoostingClassifier()
gradientBoosting.fit(train, y_train)
Y_pred = gradientBoosting.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00       1.0
          20       0.00      0.00      0.00       0.0
          24       0.00      0.00      0.00       1.0
          31       0.00      0.00      0.00       0.0
          33       0.00      0.00      0.00       1.0
          43       0.00      0.00      0.00       1.0
          65       0.00      0.00      0.00       1.0
          66       0.00      0.00      0.00       1.0
          71       0.00      0.00      0.00       0.0
          77       0.00      0.00      0.00       1.0
          85       0.00      0.00      0.00       1.0
          87       0.00      0.00      0.00       1.0
          96       0.00      0.00      0.00       0.0
          98       0.00      0.00      0.00       1.0
         100       0.00      0.00      0.00       1.0
         120       0.00      0.00      0.00       1.0
         130       0.00      0.00      0.00       1.0
         131       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Support Vector Machine 

In [0]:
svm_clf1 = SVC()
svm_clf1.fit(train, y_train)
Y_pred = svm_clf1.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00       1.0
          24       0.00      0.00      0.00       1.0
          33       0.00      0.00      0.00       1.0
          43       0.00      0.00      0.00       1.0
          65       0.00      0.00      0.00       1.0
          66       0.00      0.00      0.00       1.0
          77       0.00      0.00      0.00       1.0
          85       0.00      0.00      0.00       1.0
          87       0.00      0.00      0.00       1.0
          98       0.00      0.00      0.00       1.0
         100       0.00      0.00      0.00       1.0
         120       0.00      0.00      0.00       1.0
         130       0.00      0.00      0.00       1.0
         131       0.00      0.00      0.00       0.0
         132       0.00      0.00      0.00       0.0
         134       0.00      0.00      0.00       1.0
         136       0.00      0.00      0.00       1.0
         141       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM with Linear kernel

In [0]:
svm_clf2 = SVC(kernel='linear', C=10)
svm_clf2.fit(train, y_train)
Y_pred = svm_clf2.predict(test)
print(classification_report(y_test,Y_pred ))

SVM with Plynomial kernel

In [0]:
poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C=5)
poly_kernel_svm_clf.fit(train, y_train)
Y_pred = poly_kernel_svm_clf.predict(test)
print(classification_report(y_test,Y_pred ))

### SVM with RBF kernel

In [0]:
rbf_kernel_svm_clf = SVC(kernel="rbf", gamma=0.1, C=1000)
rbf_kernel_svm_clf.fit(train, y_train)
Y_pred = rbf_kernel_svm_clf.predict(test)
print(classification_report(y_test,Y_pred ))

### Logistic Regression

In [13]:
logreg = LogisticRegression()
logreg.fit(train, y_train)
Y_pred = logreg.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00       1.0
          24       0.00      0.00      0.00       1.0
          33       0.00      0.00      0.00       1.0
          43       0.00      0.00      0.00       1.0
          65       0.00      0.00      0.00       1.0
          66       0.00      0.00      0.00       1.0
          77       0.00      0.00      0.00       1.0
          85       0.00      0.00      0.00       1.0
          87       0.00      0.00      0.00       1.0
          98       0.00      0.00      0.00       1.0
         100       0.00      0.00      0.00       1.0
         120       0.00      0.00      0.00       1.0
         130       0.00      0.00      0.00       1.0
         134       0.00      0.00      0.00       1.0
         136       0.00      0.00      0.00       1.0
         141       0.00      0.00      0.00       1.0
         159       0.00      0.00      0.00       1.0
         166       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in

### GaussianNB

In [14]:
gaussian = GaussianNB()
gaussian.fit(train, y_train)
Y_pred = gaussian.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00         1
          24       0.00      0.00      0.00         1
          33       0.00      0.00      0.00         1
          43       0.00      0.00      0.00         1
          65       0.00      0.00      0.00         1
          66       0.00      0.00      0.00         1
          77       0.00      0.00      0.00         1
          85       0.00      0.00      0.00         1
          86       0.00      0.00      0.00         0
          87       0.00      0.00      0.00         1
          98       0.00      0.00      0.00         1
         100       0.00      0.00      0.00         1
         120       0.00      0.00      0.00         1
         130       0.00      0.00      0.00         1
         131       0.00      0.00      0.00         0
         132       0.00      0.00      0.00         0
         134       0.00      0.00      0.00         1
         136       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Decision Tree (default parameters)

In [15]:
decision_tree1 = DecisionTreeClassifier()
decision_tree1.fit(train, y_train)
Y_pred = decision_tree1.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00       1.0
          21       0.00      0.00      0.00       0.0
          24       0.00      0.00      0.00       1.0
          25       0.00      0.00      0.00       0.0
          33       0.00      0.00      0.00       1.0
          43       0.00      0.00      0.00       1.0
          48       0.00      0.00      0.00       0.0
          65       0.00      0.00      0.00       1.0
          66       0.00      0.00      0.00       1.0
          76       0.00      0.00      0.00       0.0
          77       0.00      0.00      0.00       1.0
          85       0.00      0.00      0.00       1.0
          87       0.00      0.00      0.00       1.0
          96       0.00      0.00      0.00       0.0
          98       0.00      0.00      0.00       1.0
         100       0.00      0.00      0.00       1.0
         104       0.00      0.00      0.00       0.0
         120       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Decision tree 2

In [0]:
decision_tree2 = DecisionTreeClassifier(max_depth=3, min_samples_split=0.1, min_samples_leaf=0.1)
decision_tree2.fit(train, y_train)
Y_pred = decision_tree2.predict(test)
print(classification_report(y_test,Y_pred ))

### Decision tree 3 

In [16]:
decision_tree3 = DecisionTreeClassifier(max_depth=10, min_samples_split=3,min_samples_leaf=5) 
decision_tree3.fit(train, y_train)
Y_pred = decision_tree3.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00       1.0
          20       0.00      0.00      0.00       0.0
          24       0.00      0.00      0.00       1.0
          33       0.00      0.00      0.00       1.0
          43       0.00      0.00      0.00       1.0
          65       0.00      0.00      0.00       1.0
          66       0.00      0.00      0.00       1.0
          76       0.00      0.00      0.00       0.0
          77       0.00      0.00      0.00       1.0
          85       0.00      0.00      0.00       1.0
          87       0.00      0.00      0.00       1.0
          98       0.00      0.00      0.00       1.0
         100       0.00      0.00      0.00       1.0
         120       0.00      0.00      0.00       1.0
         130       0.00      0.00      0.00       1.0
         131       0.00      0.00      0.00       0.0
         134       0.00      0.00      0.00       1.0
         136       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### RandomForestClassifier

In [20]:
random_forest = RandomForestClassifier(criterion = "gini",  max_depth= 6, n_estimators=100, max_features='log2',)

random_forest.fit(train, y_train)
Y_pred = random_forest.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00       1.0
          20       0.00      0.00      0.00       0.0
          24       0.00      0.00      0.00       1.0
          25       0.00      0.00      0.00       0.0
          33       0.00      0.00      0.00       1.0
          43       0.00      0.00      0.00       1.0
          65       0.00      0.00      0.00       1.0
          66       0.00      0.00      0.00       1.0
          71       0.00      0.00      0.00       0.0
          76       0.00      0.00      0.00       0.0
          77       0.00      0.00      0.00       1.0
          78       0.00      0.00      0.00       0.0
          85       0.00      0.00      0.00       1.0
          87       0.00      0.00      0.00       1.0
          98       0.00      0.00      0.00       1.0
         100       0.00      0.00      0.00       1.0
         113       0.00      0.00      0.00       0.0
         120       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Bagging classifier Decision tree

In [21]:
bag_clf_DT=BaggingClassifier(DecisionTreeClassifier(),n_estimators=100,max_samples=100,bootstrap=True,n_jobs=-1)
bag_clf_DT.fit(train, y_train)
Y_pred = bag_clf_DT.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00       1.0
          20       0.00      0.00      0.00       0.0
          24       0.00      0.00      0.00       1.0
          25       0.00      0.00      0.00       0.0
          33       0.00      0.00      0.00       1.0
          43       0.00      0.00      0.00       1.0
          48       0.00      0.00      0.00       0.0
          65       0.00      0.00      0.00       1.0
          66       0.00      0.00      0.00       1.0
          71       0.00      0.00      0.00       0.0
          76       0.00      0.00      0.00       0.0
          77       0.00      0.00      0.00       1.0
          85       0.00      0.00      0.00       1.0
          87       0.00      0.00      0.00       1.0
          96       0.00      0.00      0.00       0.0
          98       0.00      0.00      0.00       1.0
         100       0.00      0.00      0.00       1.0
         104       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Bagging classifier Gaussian NB


In [0]:
bag_clf_NB=BaggingClassifier(GaussianNB() ,n_estimators=100,max_samples=100,bootstrap=True,n_jobs=-1)
bag_clf_NB.fit(train, y_train)
Y_pred = bag_clf_NB.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

           1       0.75      1.00      0.86         6
           2       1.00      0.82      0.90        11
           3       0.77      1.00      0.87        10
           4       1.00      0.86      0.92        28
           5       0.96      0.66      0.78        41
           6       0.42      1.00      0.59        10
           7       1.00      1.00      1.00         7

    accuracy                           0.82       113
   macro avg       0.84      0.90      0.85       113
weighted avg       0.90      0.82      0.84       113



### Ada Boost Classifier

In [0]:
Ada_boost = AdaBoostClassifier( n_estimators= 400, learning_rate= 0.05, algorithm= 'SAMME.R')                         
Ada_boost.fit(train, y_train)
Y_pred = Ada_boost.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00        11
           3       0.37      1.00      0.54        10
           4       1.00      1.00      1.00        28
           5       0.71      1.00      0.83        41
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00         7

    accuracy                           0.70       113
   macro avg       0.30      0.43      0.34       113
weighted avg       0.54      0.70      0.60       113



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Gradiant boosting with grid search cv 


In [0]:
gradientBoosting = GradientBoostingClassifier(min_samples_leaf= 40, n_estimators= 20, min_samples_split= 200, learning_rate= 1, max_depth= 7)                                       
gradientBoosting.fit(train, y_train)
Y_pred = gradientBoosting.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

           1       0.33      0.50      0.40         6
           2       0.62      0.45      0.53        11
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        28
           5       0.75      0.98      0.85        41
           6       0.75      0.30      0.43        10
           7       1.00      0.14      0.25         7

    accuracy                           0.80       113
   macro avg       0.78      0.62      0.64       113
weighted avg       0.82      0.80      0.77       113



### Naive Bayes

In [17]:
NB1 = GaussianNB()
NB1.fit(train, y_train)
Y_pred = NB1.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00         1
          24       0.00      0.00      0.00         1
          33       0.00      0.00      0.00         1
          43       0.00      0.00      0.00         1
          65       0.00      0.00      0.00         1
          66       0.00      0.00      0.00         1
          77       0.00      0.00      0.00         1
          85       0.00      0.00      0.00         1
          86       0.00      0.00      0.00         0
          87       0.00      0.00      0.00         1
          98       0.00      0.00      0.00         1
         100       0.00      0.00      0.00         1
         120       0.00      0.00      0.00         1
         130       0.00      0.00      0.00         1
         131       0.00      0.00      0.00         0
         132       0.00      0.00      0.00         0
         134       0.00      0.00      0.00         1
         136       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### MLP Classifier

In [18]:
from sklearn.neural_network import MLPClassifier 
mlp = MLPClassifier()
mlp.fit(train, y_train)
Y_pred = mlp.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

          14       0.00      0.00      0.00       1.0
          24       0.00      0.00      0.00       1.0
          33       0.00      0.00      0.00       1.0
          43       0.00      0.00      0.00       1.0
          65       0.00      0.00      0.00       1.0
          66       0.00      0.00      0.00       1.0
          77       0.00      0.00      0.00       1.0
          85       0.00      0.00      0.00       1.0
          87       0.00      0.00      0.00       1.0
          98       0.00      0.00      0.00       1.0
         100       0.00      0.00      0.00       1.0
         120       0.00      0.00      0.00       1.0
         130       0.00      0.00      0.00       1.0
         134       0.00      0.00      0.00       1.0
         136       0.00      0.00      0.00       1.0
         141       0.00      0.00      0.00       1.0
         159       0.00      0.00      0.00       1.0
         166       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Voting Classifier

In [19]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
 estimators=[('lr', logreg), ('rf', random_forest), ('svc', svm_clf2), ('DT',decision_tree1),('NB',gaussian)],
 voting='hard'
 )
voting_clf.fit(train, y_train)
Y_pred = voting_clf.predict(test)
print(classification_report(y_test,Y_pred ))

NameError: ignored

In [0]:
from sklearn.base import clone


class OrdinalClassifier():
    
    def __init__(self, clf):
        self.clf = clf
        self.clfs = {}
    
    def fit(self, X, y):
        self.unique_class = np.sort(np.unique(y))
        if self.unique_class.shape[0] > 2:
            for i in range(self.unique_class.shape[0]-1):
                # for each k - 1 ordinal value we fit a binary classification problem
                binary_y = (y > self.unique_class[i]).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf
    
    def predict_proba(self, X):
        clfs_predict = {k:self.clfs[k].predict_proba(X) for k in self.clfs}
        predicted = []
        for i,y in enumerate(self.unique_class):
            if i == 0:
                # V1 = 1 - Pr(y > V1)
                predicted.append(1 - clfs_predict[y][:,1])
            elif y in clfs_predict:
                # Vi = Pr(y > Vi-1) - Pr(y > Vi)
                 predicted.append(clfs_predict[y-1][:,1] - clfs_predict[y][:,1])
            else:
                # Vk = Pr(y > Vk-1)
                predicted.append(clfs_predict[y-1][:,1])
        return np.vstack(predicted).T
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

### Ordinal Regreesion

In [0]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from mord import LogisticAT
from mord import OrdinalRidge
class mord.OrdinalRidge(alpha=1.0, fit_intercept=True, normalize=False, copy_X=True, max_iter=None, tol=0.001, solver='auto')

logreg = LogisticAT()
logreg.fit(train, y_train)
Y_pred = logreg.predict(test)
print(classification_report(y_test,Y_pred ))
#http://rikunert.com/ordinal_rating
#https://statistics.laerd.com/spss-tutorials/ordinal-regression-using-spss-statistics.php